In [21]:
import threading
import time
import tkinter as tk
from win10toast import ToastNotifier #pip install win10toast
from playsound import playsound #pip install playsound
from pygame import mixer
import concurrent.futures

mixer.init()
mixer.music.load("alarm.mp3")

In [67]:
class CountdownTimer:

    def __init__(self):
        self.root = tk.Tk()
        self.root.geometry("460x260")
        self.root.configure(background="green", cursor="dot blue")
        self.root.title("Countdown Timer")

        self.time_entry = tk.Entry(self.root, font=("Candara", 30), background="black", foreground="green", insertbackground="red")
        self.time_entry.grid(row=0, column=0, columnspan=2, padx=5, pady=5)

        self.start_button = tk.Button(self.root, font=("Candara", 30), text="Start", background="black", foreground="green", command=self.start_thread)
        # why thread?, or else it will block the UI, stop the counter when moving the GUI
        self.start_button.grid(row=1, column=0, padx=5, pady=5)

        self.stop_button = tk.Button(self.root, font=("Candara", 30), text="Stop", background="black", foreground="green", command=self.stop)
        self.stop_button.grid(row=1, column=1, padx=5, pady=5)

        self.clear_Text = tk.Button(self.root, font=("Candara", 30), text="Reset", background="black", foreground="green", command=lambda: self.time_entry.delete(0,tk.END))
        self.clear_Text.grid(row=2, column=0, padx=5, pady=5)

        self.time_label = tk.Label(self.root, font=("Candara", 30), text="Time: 00:00:00", background="black", foreground="green")
        self.time_label.grid(row=2, column=1, columnspan=2, padx=5, pady=5)

        self.stop_loop = False

        self.root.mainloop()
    
    def start_thread(self):
        t = threading.Thread(target=self.start)
        t.start()

    def start(self):
        self.stop_loop = False

        hours, minutes, seconds = 0,0,0
        string_split = self.time_entry.get().split(":")
        if len(string_split) == 3:
            hours = int(string_split[0])
            minutes = int(string_split[1])
            seconds = int(string_split[2])

        elif len(string_split) == 2:
            minutes = int(string_split[0])
            seconds = int(string_split[1])

        elif len(string_split) == 1:
            seconds = int(string_split[0])
        
        else:
            print("Invalid Time format!")
            return

        full_seconds = hours * 3600 + minutes * 60 + seconds

        while full_seconds > 0 and not self.stop_loop:
            full_seconds -= 1

            minutes, seconds = divmod(full_seconds, 60)
            hours, minutes = divmod(minutes, 60)

            self.time_label.config(text=f"Time: {hours:02d}:{minutes:02d}:{seconds:02d}")
            self.root.update()
            time.sleep(1)

        if not self.stop_loop:
            def notify():
                toast = ToastNotifier()
                toast.show_toast("CountdownTimer", "Time is up!", duration=10)

            #play sound
            def sound():
                #playsound("alarm.mp3")
                mixer.music.play()

            with concurrent.futures.ThreadPoolExecutor() as executor:
                f1 = executor.submit(notify)
                f2 = executor.submit(sound)
            
            

    def stop(self):
        self.stop_loop = True
        self.time_label.config(text="Time: 00:00:00")
        self.time_entry.delete(0,tk.END)
        mixer.music.stop()



In [69]:
CountdownTimer()